In [1]:
using YAML
using DataFrames
using CSV

In [25]:
headers = ["num_iterations","learning_rate","feature_fraction","min_data_in_leaf","num_leaves","max_depth","lambda_l1","lambda_l2","min_gain_to_split","bagging_fraction"]
experimentos = ("KA4940_14_ShortNotPre","KA4940_14_ShortPre1","KA4940_13","KA4949","KA4948","KA4947","KA4946.1","KA4945","KA4944","KA4940_01_long","KA4942.1","KA4941","KA4940")
envios= (10000:500:12000)


10000:500:12000

In [3]:
path = "buckets/b1/exp/"

"buckets/b1/exp/"

In [10]:
scores_df = DataFrame(CSV.File("/home/l_hermannsauer/resultados.csv"));

In [71]:
resultados = DataFrame(
    alumno = String[],
    semilla = Int[],
    num_iterations = Int[],
    learning_rate = Float64[],
    feature_fraction = Float64[],
    min_data_in_leaf = Int[],
    num_leaves = Int[],
    max_depth = Int[],
    lambda_l1 = Float64[],
    lambda_l2 = Float64[],
    min_gain_to_split = Float64[],
    bagging_fraction = Float64[],
    envios = Int[],
    score = String[]
)

Row,alumno,semilla,num_iterations,learning_rate,feature_fraction,min_data_in_leaf,num_leaves,max_depth,lambda_l1,lambda_l2,min_gain_to_split,bagging_fraction,envios,score
,String,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Int64,String


In [72]:
resultados

Row,alumno,semilla,num_iterations,learning_rate,feature_fraction,min_data_in_leaf,num_leaves,max_depth,lambda_l1,lambda_l2,min_gain_to_split,bagging_fraction,envios,score
,String,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Int64,String


In [76]:
function extract_params_and_score(
    df::DataFrame,
    folder_name::String,
    submission::Integer,
    scores_df::DataFrame,
    yaml_keys::Vector{String}
)    
    # Read the YAML file
    yml_path = joinpath("/home/l_hermannsauer/buckets/b1/exp/exp$(folder_name[3:end])", "PARAM.yml")
    
    if !isfile(yml_path)
        #@warn("Not found $yml_path")
        return
    end
    params = YAML.load_file(yml_path)

    # Extract each requested YAML parameter
    extracted_params = Dict{String, Any}()
    for key in yaml_keys
        # Split nested keys (e.g., "lgbm.param_fijos.boosting" → ["lgbm", "param_fijos", "boosting"])
        
        try
            value = params["out"]["lgbm"]["mejores_hiperparametros"][key]
            extracted_params[key] = value
        catch e 
            value = params["lgbm"]["param_fijos"][key]
            extracted_params[key] = value

        end
    end


    # Get the matching score from the scores dataframe
    matching_rows = filter(row -> row.Exp == folder_name && row.Envios == submission, scores_df)
    if isempty(matching_rows)
        error("No matching score found for Exp: $folder_name, Envios: $submission")
    end
    puntaje = matching_rows[1, :Puntaje]

    extracted_params["alumno"] = "Leandro Hermann"
    extracted_params["semilla"] = 106109
    extracted_params["score"] = puntaje
    extracted_params["envios"] = submission
    push!(df, extracted_params)
end

extract_params_and_score (generic function with 2 methods)

In [77]:
for exp in experimentos
    for envio in envios
        extract_params_and_score(resultados,exp, envio, scores_df, headers)
    end
end

In [78]:
CSV.write("resultados.csv", resultados)


"resultados.csv"